In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
# Set default plotly theme
pio.templates.default = 'plotly_white'

from src.utilities import Matrix

from src.observables import *

SUFFIX = '.npy'
OMEGA = 1  # Interaction Strength

HEAT = True
TIMES = False  # For stochastic interaction times

### Functions

In [ ]:
# Files utilities

def file_dims(filename):
    return int(filename.split('_')[-3][1:])


def file_time(filename):
    return int(filename.split('_')[-2][1:])


def file_timedelta(filename):
    return float(filename.split('_')[-1][2:])


def check_file_metadata(filename, d, dt):
    dims = file_dims(filename)
    timedelta = file_timedelta(filename)
    return dims == d and timedelta == dt


def get_all_files(state, key, observable):
    files = [file for file in os.listdir(f'../objects/saved{state}_{key}') if file.endswith(SUFFIX) and file.startswith(f'rho_{observable}')]
    return files

def get_last_file(startswith, dims, state, key):
    all_files = [f.removesuffix(SUFFIX) for f in os.listdir(f'../objects/saved/{state}_{key}') if f.startswith(f'{startswith}')]
    timedeltas = [file_timedelta(f) for f in all_files]
    max_times = {dt: max([file_time(f) for f in all_files if check_file_metadata(f, dims, dt)]) for dt in timedeltas}
    files = {dt: [f for f in all_files if check_file_metadata(f, dims, dt) and file_time(f) == max_times[dt]][0] for dt in timedeltas}
    return dict(sorted(files.items()))

def get_cov_of_dims(dims, state, key, filtered_keys=None) -> dict:
    files = get_last_file(f'rho_covariance_D{dims}', dims, state, key)
    if filtered_keys is not None:
        files = {k: files[k] for k in filtered_keys}
    return files

def get_heat_of_dims(dims, state, key, filtered_keys=None) -> dict:
    files = get_last_file(f'rho_heats_D{dims}', dims, state, key)
    if filtered_keys is not None:
        files = {k: files[k] for k in filtered_keys}
    return files

def make_file_name(code, observable):
    return f'../objects/saved/{code}_rho_{observable}.npy'

#=======================================================================================================================

def single_entropy(cov, system):
    d1, d2 = symplectic_eigenvalues(cov)
    if system == 1:
        x1 = d1 + 1/2
        x2 = d1 - 1/2
    elif system == 2:
        x1 = d2 + 1/2
        x2 = d2 - 1/2
    return x1 * np.log(x1) - x2 * np.log(x2)

def get_observable(observable, covs, iterator):
    if observable == 'Entropy':
        func = symplectic_entropy
    elif observable == 'Mutual Information':
        func = mutual_information
    elif observable == 'Quantum Discord':
        func = gaussian_quantum_discord
    elif observable == 'Logarithmic Negativity':
        func = logarithmic_negativity
    else:
        raise ValueError(f'Observable {observable} not supported')

    arr = []
    for i in iterator:
        arr.append(func(covs[i]))
    return np.array(arr).real

#====================================================================

def stable_temperature(a, p):
    a = float(a)
    p = float(p)
    ga = 2*a**2
    gb = (1-a**2) * (1+np.cos(p))
    return - 1 / np.log(ga / gb)

# Effects of Interaction Time on the evolution
For the paper we inspect:
1. the difference in the evolution of the observables for different interaction times.
2. the difference of the partial evolution of observables for different interaction times.
3. exploration of the alpha-phi space fixing the stable temperature

For (1) use data with IDs [032, 033, 034, 037] and for (2) use data with IDs [035, 036]. For (3) we use ['T11', 'T12', 'T13', 'T14', 'T15'] with stable T=1 and ['T21', 'T22', 'T23', 'T24', 'T25'] with stable T=2.

In [ ]:
log = pd.read_csv('../objects/saved/logs.csv', sep=',', decimal='.', index_col='Id')
# Format the index values as strings with leading zeros
log.index = log.index.astype(str).str.zfill(3)

# Filter the data to load
filter_dict = {
    # 'state': 'thermal',
    # 'dimensions': 17,
    # 'timedelta': [1.0, 0.5, 0.1],
    # 'omega': 0.5,
}
id_T1_dt10 = ['T11', 'K12', 'T13', 'T14', 'T15']
id_T1_dt05 = ['T16', 'T17', 'T18', 'T19', 'T110']
id_T1_dt01 = ['T111', 'T112', 'T113', 'T114', 'T115']
id_T2_dt10 = ['T21', 'T22', 'T23', 'T24', 'T25']
id_T2_dt05 = ['T26', 'T27', 'T28', 'T29', 'T210']
id_T2_dt01 = ['T211', 'T212', 'T213', 'T214', 'T215']
filter_ids = id_T1_dt10 + id_T1_dt05 + id_T1_dt01 + id_T2_dt10 + id_T2_dt05 + id_T2_dt01
# filter_ids = [f"R{i}{j}" for i in range(1, 6) for j in ['01', '05', '10']]
filter_ids = ['032', '034']

# Convert the filter conditions to a query string
query_str = " and ".join(
    [
        f"{key} == {value!r}" if not isinstance(value, list) 
        else f"{key} in {value!r}" for key, value in filter_dict.items()
    ]
)

# Filter the DataFrame using the query method and the filtered IDs
files_to_load = log.query(query_str).loc[filter_ids] if query_str else log.loc[filter_ids]
print(f"There are {len(files_to_load)} files to load")

In [ ]:
observables = ['Temperatures', 'Entropies', 'Mutual Information', 'Quantum Discord', 'Logarithmic Negativity', 'Heat Flux']
df = pd.DataFrame(columns=observables)
steps_per_timedelta = 1

distribution = 'gaussian'

steps = 10000  # To filter data points

print(f'Loading files with:\n'
      f'{"Evolution Steps":>21}\t'
      f'{"Log ID":>10}\t'
      f'{"Interaction Time":>21}\t'
      f'{"Interaction Strength":>21}\t')
for indx, metadata in files_to_load.iterrows():
    dt_df = pd.DataFrame(columns=observables)
    timedelta = metadata['timedelta']
    omega = metadata['omega']
    
    cov_filename = f'../objects/saved/{indx}_rho_covariance.npy'
    heat_filename = f'../objects/saved/{indx}_rho_heats.npy'
    times_filename = f'../objects/saved/{indx}_rho_{distribution}_times.npy'
    cov_evolution = np.load(cov_filename)
    heat_transfer = np.load(heat_filename).real if HEAT else np.zeros((len(cov_evolution),3))
    
    print(f'{len(cov_evolution):>21}\t'
          f'{indx:>10}\t'
          f'{timedelta:>21}\t'
          f'{omega:>21}')
    
    # Iterators to select only a sample of data
    iterator = range(len(cov_evolution) - 1)  # np.linspace(0, len(cov_evolution) - 1, steps).astype(int)
    heat_iterator = range(len(cov_evolution) - 1)  # np.linspace(0, len(heat_transfer) - 1, steps).astype(int)
    interaction_times = np.load(times_filename) if TIMES else [timedelta for i in iterator]
    time = np.cumsum(interaction_times) if TIMES else [i * timedelta for i in iterator]
    
    dt_df['Log ID'] = [indx for _ in iterator]
    dt_df['Time'] = time
    dt_df['Interaction Time'] = interaction_times 
    dt_df['d1'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d2'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d1-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d2-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real

    for obs in observables:
        if obs == 'Heat Flux':
            dt_df['J1'] = np.array([heat_transfer[i, 0] for i in heat_iterator])
            dt_df['J2'] = np.array([heat_transfer[i, 1] for i in heat_iterator])
            dt_df['Jc'] = np.array([heat_transfer[i, 2] for i in heat_iterator])
        elif obs == 'Entropies':
            dt_df['S'] = np.array([symplectic_entropy(cov_evolution[i]) for i in iterator]).real
            dt_df['S1'] = np.array([single_entropy(cov_evolution[i], 2) for i in iterator]).real
            dt_df['S2'] = np.array([single_entropy(cov_evolution[i], 1) for i in iterator]).real
        elif obs == 'Temperatures':
            dt_df['N1'] = np.array([mean_photon_numbers(cov_evolution[i])[0] for i in iterator]).real
            dt_df['N2'] = np.array([mean_photon_numbers(cov_evolution[i])[1] for i in iterator]).real
        elif obs == 'Quantum Discord':
            dt_df[obs] = get_observable(obs, cov_evolution, iterator).tolist()
        else:
            dt_df[obs] = get_observable(obs, cov_evolution, iterator)

    df = pd.concat([df, dt_df])
df.drop('Heat Flux', axis=1, inplace=True)
df.drop('Entropies', axis=1, inplace=True)
df.drop('Temperatures', axis=1, inplace=True)

In [ ]:
fig, ax = plt.subplots()
for indx, group in df.groupby('Log ID'):
    ax.plot(group['Time'], label=indx)

In [ ]:
dt_df['Quantum Discord']

In [ ]:
fig_n, ax = plt.subplots()

for indx, group in df.groupby('Log ID'):
    timedelta = log.loc[indx, 'timedelta']
    temp = stable_temperature(log.loc[indx, 'alpha'], log.loc[indx, 'phi'])
    label = f"T = {temp} | $\phi$ = {log.loc[indx, 'phi']}"
    ax.plot(group['Time'], group['N1'], label=label)
    ax.plot(group['Time'], group['N2'], label=label)
ax.legend()
fig_n.suptitle('<N>')
ax.set_xlim(0, 50000)
# ax.axhline(0.58, color='black', ls='--')

In [ ]:
def get_temperature(n):
    beta = np.log(1 / n + 1)
    return 1 / beta

final_n2 = df.loc[df['Log ID'] == 'T113']['N1'].iloc[-1]
get_temperature(final_n2)

In [ ]:
from src.utilities import set_matplotlib_defaults
save = False
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12, 8))
set_matplotlib_defaults()

for i, (indx, group) in enumerate(df.groupby('Log ID')):
    timedelta = log.loc[indx, 'timedelta']
    temp = stable_temperature(log.loc[indx, 'alpha'], log.loc[indx, 'phi'])
    label = f"T = {temp} | $\phi$ = {log.loc[indx, 'phi']}"
    ax1.plot(group['Time'], group['Quantum Discord'],ls='-', label=label)
    ax2.plot(group['Time'], group['S'],ls='-', label=label)
    ax1.set_xlim(0, 60)
    ax1.set_ylim(0, 0.0005)
    ax2.set_xlim(0, 200)
    # ax2.set_ylim(3.0, 3.5)
    ax1.set_ylabel('Quantum Discord')
    ax2.set_ylabel('Entropy')
    ax2.set_xlabel('Time (a.u.)')
    ax1.legend()
    if save:
        fig.savefig(f'../img/obs_evolution/thermal/video/quantum_discord_frame_{timedelta}.png', dpi=300)
        ax1.clear()
        ax2.clear()


In [ ]:
df['Labels'] = df['Log ID'].apply(lambda x: f"ID{x}: T = {stable_temperature(log.loc[x, 'alpha'], log.loc[x, 'phi'])} | phi = {log.loc[x, 'phi']}")
df_melt = df.melt(id_vars=['Time', 'Labels'], value_vars=['N1', 'N2'], var_name='System', value_name='Temperature')
fig_temp = px.line(df_melt, x='Time', y='Temperature',
                   color='Labels', line_dash='System',
                   title='Mean Photon Numbers in the Systems',
                   labels={'Temperature': 'Mean Photon '
                                          'Number', 'Time': 'Time (a.u.)'})
fig_temp.update_xaxes(range=[0, 1500])
fig_temp.show()

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Log ID'], value_vars=['S1', 'S2'], var_name='System', value_name='Entropy')
fig_entropy = px.line(df_melt, x='Time', y='Entropy',
                   color='Log ID', line_dash='System',
                   title='Systems Entropies')
fig_entropy.update_xaxes(range=[0, 1500])
fig_entropy.show()

In [ ]:
df['T_ratio'] = df['N2'] / df['N1']

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
reds = plt.cm.Reds(np.linspace(0.2, 1, len(df['Log ID'].unique())))
blues = plt.cm.Blues(np.linspace(0.2, 1, len(df['Log ID'].unique())))

for i, (name, group) in enumerate(df.groupby('Log ID')):
    label = log.loc[name, 'timedelta']
    ax1.plot(group['Time'], group['T_ratio'], label=label, ls='--', color=reds[i])
    ax2.plot(group['Time'], group['Mutual Information'], label=label, ls='-', color=blues[i])

ax1.set_ylabel('T_ratio')
ax2.set_ylabel('Quantum Discord', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

ax1.set_xlim(0, 50)

ax1.legend(loc='upper center')
ax2.legend(loc='upper right')
fig.tight_layout()
plt.show()


In [ ]:
fig_discord = px.line(df, x='Time', y='Quantum Discord', color='Log ID', title='Quantum Discord')
fig_discord.update_xaxes(range=[0, 200])
fig_discord.show()

In [ ]:
fig_mutinf = px.line(df, x='Time', y='Mutual Information', color='Interaction Time', title='Mutual Information')
fig_mutinf.update_xaxes(range=[0, 1500])
fig_mutinf.show()

In [ ]:
fig_logneg = px.line(df, x='Time', y='Logarithmic Negativity', color='Interaction Time', title='Logarithmic Negativity')
fig_logneg.update_xaxes(range=[0, 1500])
fig_logneg.show()

In [ ]:
fig_temp = px.line(df, x='Time', y='Jc', 
                   color='Log ID', line_dash='Log ID',
                   title='Total Heat Current')
fig_temp.show()

In [ ]:
df_melt = df.loc[df['Time']>0].melt(id_vars=['Time', 'Log ID'], value_vars=['J1', 'J2', 'Jc'], var_name='Current', value_name='Heat')
fig_heat = px.line(df_melt, x='Time', y='Heat',
                   color='Log ID', line_dash='Current',
                   title='Heat Currents')
fig_heat.update_xaxes(range=[0, 100])
fig_heat.update_yaxes(range=[-0.005, 0.1])
fig_heat.show()

## Symplectic Eigenvalues

In [ ]:
fig, ax = plt.subplots()
x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d1']
y2=df.loc[df['Interaction Time'] == 1.0]['d1-']
ax.plot(x, y1, y2)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d2']
y2=df.loc[df['Interaction Time'] == 1.0]['d2-']

ax.plot(x, y1, y2)
plt.show()

## Save Data Frame

In [ ]:
df

In [ ]:
pd.to_pickle(df, f'../objects/saved/observables.pkl')